In [1]:
import pandas as pd
from functools import reduce

In [2]:
# Extraction
# Food nutrients
file_name = '/Users/Orchestrator/Desktop/DietGenerator/food_stats.xlsx'
data_raw = pd.read_excel(file_name, sheet_name=None)
# DRI
dosage = '/Users/Orchestrator/Desktop/DietGenerator/recommended_intake.xlsx'
data_dri = pd.read_excel(dosage, sheet_name='DRI')

In [3]:
age   = 25
kcald = 2000

split = {'Meat':       0.25,
         'Vegetables': 0.5,
         'Fruit':      0.25}

male      = True
pregnant  = False
lactation = False

if male:        group = 'Males'
elif pregnant:  group = 'Pregnancy'
elif lactation: group = 'Lactation'
else:           group = 'Female'

In [4]:
data_dri[(data_dri['Lower Age'] <= age) & (data_dri['Upper Age'] > age) & (data_dri['Group'] == group)]

,Group,Lower Age,Upper Age,Total Water L/d,Carbohydrate g/d,Total Fiber g/d,Fat g/d L,Fat g/d U,Linoleic Acid g/d,α-Linolenic Acid g/d,...,Vitamin K (μg/d),Thiamin (mg/d),Riboflavin (mg/d),Niacin (mg/d)e,Vitamin B6 (mg/d),Folate (μg/d)f,Vitamin B12 (μg/d),Pantothenic Acid (mg/d),Biotin (μg/d),Choline (mg/d)g
6,Males,19.0,30.0,3.7,130,38,20,35,17.0,1.6,...,120,1.2,1.3,16,1.3,400,2.4,5.0,30,550


In [5]:
# Processing and cleaning
data_foods = reduce(lambda left, right: pd.merge(left, right[[i for i in right.columns if i not in left.columns or i == 'Food Code']], on='Food Code'), data_raw.values())
data_foods = data_foods[(data_foods['Food Code'].notnull()) & (data_foods['Energy (kcal) (kcal)'] != 'N')].drop(['Footnote','Previous'], axis=1).apply(lambda x: x.str.replace(r"\(.*\)",""), axis=1).fillna(0).reset_index(drop=True)
data_foods.iloc[:, 5:] = data_foods.iloc[:, 5:].replace(['N', 'Tr', '0r',''], ['0'] * 4).astype(float)

# Filter dubious caloric count foods. 
data_foods = data_foods[data_foods['Energy (kcal) (kcal)'] != 0]
data_foods['Portions'] = 1 # Portion count

data_foods.head(2)

,Food Code,Food Name,Description,Group,Main data references,Edible proportion,Specific gravity,Total solids,Nitrogen conversion factor,Glycerol conversion factor,...,Beta-sitosterol (mg),Brassicasterol (mg),Campesterol (mg),Delta-5-avenasterol (mg),Delta-7-avenasterol (mg),Delta-7-stigmastenol (mg),Stigmasterol (mg),Citric acid (g),Malic acid (g),Portions
0,13-145,"Ackee, canned, drained",8 cans,DG,"MW4, 1978; and Vegetables, Herbs and Spices Su...",0.0,0.0,0.0,0.0,0.0,...,4.8,0.0,0.3,0.0,0.0,0.0,12.0,0.0,0.0,1
1,13-146,"Agar, dried",Literature sources,DG,Wu Leung et al. Food composition table for us...,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [6]:
# Selection of vegetables.
data_veg = data_foods[(data_foods['Group'] == 'DG') & (data_foods['Food Name'].str.contains('raw'))].reset_index(drop=True)
# Cleaning briefly the data. We get the max energy values for similar names.
data_veg['Food Name'] = data_veg.apply(lambda x: x['Food Name'].split(',')[0], axis=1)
data_veg.groupby('Food Name').max().reset_index(drop=True)
data_veg.head(2)

,Food Code,Food Name,Description,Group,Main data references,Edible proportion,Specific gravity,Total solids,Nitrogen conversion factor,Glycerol conversion factor,...,Beta-sitosterol (mg),Brassicasterol (mg),Campesterol (mg),Delta-5-avenasterol (mg),Delta-7-avenasterol (mg),Delta-7-stigmastenol (mg),Stigmasterol (mg),Citric acid (g),Malic acid (g),Portions
0,13-148,Alfalfa sprouts,Analytical and literature sources,DG,"Vegetables, Herbs and Spices Supplement, 1991",1.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,13-149,Amaranth leaves,Literature sources,DG,Wu Leung et al. Food composition table for us...,0.89,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [7]:
K = 0
plan_diet = pd.DataFrame()

section_kcal = kcald * split['Vegetables']

while(K < section_kcal):
    sample = data_veg.sample(1)
    sample_kcal = sample['Energy (kcal) (kcal)'].values
    
    if (section_kcal - sample_kcal) <= 50:
        break
        
    plan_diet = plan_diet.append(sample)
    K += sample_kcal
    
print('Recommended diet:')
plan_diet.groupby(['Food Name']).sum()[['Energy (kcal) (kcal)', 'Portions']]

Recommended diet:


,Energy (kcal) (kcal),Portions
Food Name,,
Amaranth leaves,18.0,1
Arrowhead,107.0,1
Beetroot,36.0,1
Breadfruit,95.0,1
Broccoli,34.0,1
Cabbage,24.0,1
Carrots,30.0,1
Celery,7.0,1
Courgette,18.0,1
